<a href="https://colab.research.google.com/github/imnoteinstien/1skills-introduction-to-github/blob/main/PharmaAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4 langchain chromadb requests pandas sqlalchemy pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 17.1 MB/s eta 

In [1]:
# Create base folder
!mkdir pharma-ai-chatbot

# Create subfolders
!mkdir pharma-ai-chatbot/app
!mkdir pharma-ai-chatbot/data
!mkdir pharma-ai-chatbot/models

In [2]:
# Basic requirements
!pip install openai python-dotenv gradio

# Optional API providers
!pip install google-generativeai deepseek-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 90.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: Could not find a version that satisfies the requirement deepseek-api (from versions: none)
ERROR: No matching distribution found for deepseek-api


In [3]:
# Check folders
!ls pharma-ai-chatbot

# Check installed packages
!pip list | grep -E "openai|gradio|google-generativeai|deepseek"

app  data  models
google-generativeai                      0.8.4
gradio                                   5.13.1
gradio_client                            1.6.0
openai                                   1.59.9


In [35]:
# Create data subfolders
!mkdir -p pharma-ai-chatbot/data/fda_labels
!mkdir -p pharma-ai-chatbot/data/pubchem_compounds
!mkdir -p pharma-ai-chatbot/data/pmc_pdfs
!mkdir -p pharma-ai-chatbot/data/pmc_texts

In [76]:
%%writefile pharma-ai-chatbot/data/fda_scraper.py
import requests
import os
import time
import json
from tqdm import tqdm
from datetime import datetime
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Configuration
MAX_RETRIES = 3
BACKOFF_FACTOR = 1
BATCH_SIZE = 1000  # Max allowed by FDA API
BASE_URL = "https://api.fda.gov/drug/label.json"
SAFE_FIELDS = "openfda.generic_name,indications_and_usage,dosage_and_administration,adverse_reactions"

def create_session():
    """Create a resilient requests session with retries"""
    session = requests.Session()
    retries = Retry(
        total=MAX_RETRIES,
        backoff_factor=BACKOFF_FACTOR,
        status_forcelist=[429, 500, 502, 503, 504]
    )
    session.mount('https://', HTTPAdapter(max_retries=retries))
    return session

def get_existing_ids():
    """Get set of already downloaded label IDs"""
    return set(f.split('.')[0] for f in os.listdir('pharma-ai-chatbot/data/fda_labels'))

def scrape_fda_labels():
    os.makedirs('pharma-ai-chatbot/data/fda_labels', exist_ok=True)
    session = create_session()

    try:
        # Get total count
        count_resp = session.get(
            BASE_URL,
            params={'count': 'openfda.generic_name.exact'},
            timeout=10
        )
        total = sum(item['count'] for item in count_resp.json().get('results', []))

        existing = len(get_existing_ids())
        print(f"📊 Total labels: {total} | Existing: {existing} | New: {total - existing}")

        params = {
            'limit': BATCH_SIZE,
            'skip': 0,
            'search': 'exists:openfda.generic_name',
            'sort': 'effective_time:desc',
            'fields': SAFE_FIELDS
        }

        downloaded = 0
        with tqdm(total=total-existing, desc="📥 Downloading", unit="label") as pbar:
            while True:
                try:
                    response = session.get(BASE_URL, params=params, timeout=15)
                    response.raise_for_status()
                    data = response.json()

                    if 'results' not in data:
                        print("⚠️ API structure changed - missing 'results' key")
                        break

                    batch_ids = {label.get('id') for label in data['results']}
                    existing_ids = get_existing_ids()
                    new_labels = [label for label in data['results']
                                if label.get('id') not in existing_ids]

                    # Save batch
                    for label in new_labels:
                        filename = f"pharma-ai-chatbot/data/fda_labels/{label['id']}.json"
                        with open(filename, 'w') as f:
                            json.dump(label, f)
                        downloaded += 1
                        pbar.update(1)

                    # Progress check
                    if len(new_labels) == 0 or params['skip'] + BATCH_SIZE > 10000:
                        break  # FDA API has skip limit of 10,000

                    params['skip'] += BATCH_SIZE
                    time.sleep(0.5)  # Conservative rate limiting

                except (requests.exceptions.RequestException, KeyError) as e:
                    print(f"⚠️ Temporary error: {str(e)} - Retrying...")
                    time.sleep(5)
                    continue

        # Update metadata
        metadata = {
            'last_run': datetime.now().isoformat(),
            'total_downloaded': downloaded,
            'next_start': params['skip']
        }
        with open('pharma-ai-chatbot/data/fda_labels/metadata.json', 'w') as f:
            json.dump(metadata, f)

        print(f"✅ Successfully downloaded {downloaded} new labels")

    except Exception as e:
        print(f"❌ Critical failure: {str(e)}")

if __name__ == "__main__":
    scrape_fda_labels()

Overwriting pharma-ai-chatbot/data/fda_scraper.py


In [60]:
!pip install requests tqdm pymupdf multiprocess

In [ ]:
!python pharma-ai-chatbot/data/fda_scraper.py


📊 Total labels: 26588 | Existing: 62806 | New: -36218
📥 Downloading: 0label [00:00, ?label/s]⚠️ Temporary error: 400 Client Error: Bad Request for url: https://api.fda.gov/drug/label.json?limit=1000&skip=0&search=exists%3Aopenfda.generic_name&sort=effective_time%3Adesc&fields=openfda.generic_name%2Cindications_and_usage%2Cdosage_and_administration%2Cadverse_reactions - Retrying...
⚠️ Temporary error: 400 Client Error: Bad Request for url: https://api.fda.gov/drug/label.json?limit=1000&skip=0&search=exists%3Aopenfda.generic_name&sort=effective_time%3Adesc&fields=openfda.generic_name%2Cindications_and_usage%2Cdosage_and_administration%2Cadverse_reactions - Retrying...
⚠️ Temporary error: 400 Client Error: Bad Request for url: https://api.fda.gov/drug/label.json?limit=1000&skip=0&search=exists%3Aopenfda.generic_name&sort=effective_time%3Adesc&fields=openfda.generic_name%2Cindications_and_usage%2Cdosage_and_administration%2Cadverse_reactions - Retrying...
⚠️ Temporary error: 400 Client Err